# Plantilla para la Tarea online BDA02

# Nombre del alumno: 

En esta tarea deberás completar las celdas que están incompletas. Se muestra el resultado esperado de la ejecución. Se trata de que implementes un proceso MapReduce que produzca ese resultado. Puedes implementar el proceso MapReduce con el lenguaje y librería que prefieras (`Bash`, Python, `mrjob` ...). Los datos de entrada del proceso son meros ejemplos y el proceso que implementes debería funcionar con esos y cualquier otro fichero de entrada que tenga la misma estructura.

## 1.- Partiendo del fichero de `notas.txt`, calcula la nota más alta obtenida por cada alumno con un proceso MapReduce.

Es decir, que si tenemos el fichero de notas:

In [54]:
%%writefile notas.txt
pedro 6 7
luis 0 4
ana 7
pedro 8 1 3
ana 5 6 7
ana 10
luis 3

Overwriting notas.txt


Se espera obtener el siguiente resultado:

![solución 1](./img/1.png)

In [55]:
%%writefile mapper.sh
#!/bin/bash

# Leemos línea a línea de la entrada estándar
while read line; do
    
    # Extraemos el nombre de la línea
    name=${line%% *}
    
    # Procesamos nota a nota
    for mark in ${line#* }; do
                  
        # para cada nota emitimos nombre,nota
        echo -e "$name,$mark"
    done    
done

Overwriting mapper.sh


In [56]:
! chmod +x mapper.sh

In [57]:
! cat notas.txt | ./mapper.sh |sort

ana,10
ana,5
ana,6
ana,7
ana,7
luis,0
luis,3
luis,4
pedro,1
pedro,3
pedro,6
pedro,7
pedro,8


In [58]:
%%writefile reducer.sh
#!/bin/bash
prev_name=
acc=0
declare -i n1
declare -i n2
n1=0
n2=0
n_marks=0
# Leemos línea a línea
while read line; do
    
    # Extraemos el nombre y la nota
    name=${line%,*}
    mark=${line#*,} 
    
    if [ -z "$prev_name" -o "$prev_name" == "$name" ]; then
           n2=$mark         
           if [ "$n2" -ge "$n1" ]; then
               n1=n2
               n_marks=$n2
           else
               n_marks=$n1
           fi   
    # Cuando el nombre sea diferente, emitimos el nombre anterior,la nota más alta anterior
    else
        echo $prev_name,$n_marks
        n1=0
        n2=0
    fi
    prev_name=$name
done
           
# Emitimos el nombre y la nota más alta del último nombre
echo $prev_name,$n_marks

Overwriting reducer.sh


In [59]:
! chmod +x reducer.sh

In [60]:
! cat notas.txt | ./mapper.sh | sort | ./reducer.sh

ana,10
luis,4
pedro,8


In [61]:
! hadoop fs -rm -f -r /user/root/notas.txt /user/root/output

In [62]:
! hadoop fs -copyFromLocal notas.txt /user/root/

In [63]:
! hadoop fs -ls /user/root

Found 3 items
-rw-r--r--   3 root supergroup    2205995 2023-01-18 10:19 /user/root/2000-0.txt
-rw-r--r--   3 root supergroup         61 2023-01-18 10:37 /user/root/notas.txt
drwxr-xr-x   - root supergroup          0 2023-01-18 10:19 /user/root/tmp


In [64]:
! hadoop fs -cat /user/root/notas.txt

pedro 6 7
luis 0 4
ana 7
pedro 8 1 3
ana 5 6 7
ana 10
luis 3


In [65]:
! mapred streaming \
    -D mapred.textoutputformat.separator="," \
    -files /media/notebooks/mapper.sh,/media/notebooks/reducer.sh \
    -input /user/root/notas.txt \
    -output /user/root/output \
    -mapper mapper.sh \
    -reducer reducer.sh

packageJobJar: [] [/app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar] /tmp/streamjob2858468526024176086.jar tmpDir=null
2023-01-18 10:37:47,965 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
2023-01-18 10:37:48,219 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
2023-01-18 10:37:48,451 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1674032573936_0010
2023-01-18 10:37:48,867 INFO mapred.FileInputFormat: Total input files to process : 1
2023-01-18 10:37:48,978 INFO mapreduce.JobSubmitter: number of splits:2
2023-01-18 10:37:49,072 INFO Configuration.deprecation: mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2023-01-18 10:37:49,219 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1674032573936_0010
2023-01-18 10:37:4

In [66]:
! hadoop fs -ls /user/root/output

Found 2 items
-rw-r--r--   3 root supergroup          0 2023-01-18 10:38 /user/root/output/_SUCCESS
-rw-r--r--   3 root supergroup         25 2023-01-18 10:38 /user/root/output/part-00000


In [67]:
! hadoop fs -cat /user/root/output/*

ana,10,
luis,4,
pedro,8,


## 2.- Usando un proceso MapReduce muestra las 10 palabras más utilizadas en `El Quijote`.

Lo primero será descargar El Quijote:

In [68]:
! wget -O '2000-0.txt' https://www.gutenberg.org/files/2000/2000-0.txt

--2023-01-18 10:38:20--  https://www.gutenberg.org/files/2000/2000-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... ::ffff:152.19.134.47, 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|::ffff:152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226045 (2.1M) [text/plain]
Saving to: ‘2000-0.txt’

2000-0.txt          100%[===================>]   2.12M  2.68MB/s    in 0.8s    

2023-01-18 10:38:21 (2.68 MB/s) - ‘2000-0.txt’ saved [2226045/2226045]



Al igual que hicimos en la primera práctica, eliminamos aquellas líneas que son metadata y no forman parte de la obra. Sobrescribimos el fichero sin esas líneas.

In [69]:
with open('2000-0.txt') as f:
    lines = f.readlines()

head = 24
tail = 360
book = lines[head:-tail]

with open('2000-0.txt', 'w') as f:
    for line in book:
        f.write(f"{line}\n")


El resultado debería ser el mismo que el que obtuvimos en la primera práctica.

![solución 2](./img/2.png)

In [70]:
%%writefile frecuencyMR.py
#!/usr/bin/python3

from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
import re

WORD_RE = re.compile(r"[\w']+")


class MRMustUsedWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_frecuency)
        ]

    def mapper_get_words(self, _, line):
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        yield None, (sum(counts), word)
        
    def reducer_frecuency(self, _, counts):
        sorted_freqs = sorted(counts, key=lambda t: t[0], reverse=True)
        yield None, list(sorted_freqs[0:10])



if __name__ == '__main__':
    MRMustUsedWord.run()

Overwriting frecuencyMR.py


In [71]:
! chmod ugo+x frecuencyMR.py

In [72]:
! python3 frecuencyMR.py 2000-0.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/frecuencyMR.root.20230118.093823.327141
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/frecuencyMR.root.20230118.093823.327141/output
Streaming final output from /tmp/frecuencyMR.root.20230118.093823.327141/output...
null	[[20769, "que"], [18408, "de"], [18266, "y"], [10491, "la"], [9874, "a"], [8281, "en"], [8265, "el"], [6334, "no"], [4769, "los"], [4752, "se"]]
Removing temp directory /tmp/frecuencyMR.root.20230118.093823.327141...


In [73]:
! hadoop fs -rm -f -r /user/root/notas.txt /user/root/output

Deleted /user/root/notas.txt
Deleted /user/root/output


In [74]:
! hadoop fs -copyFromLocal 2000-0.txt /user/root/

copyFromLocal: `/user/root/2000-0.txt': File exists


In [75]:
! python3 frecuencyMR.py -r hadoop hdfs:///user/root/2000-0.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/frecuencyMR.root.20230118.093834.974091
uploading working dir files to hdfs:///user/root/tmp/mrjob/frecuencyMR.root.20230118.093834.974091/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/frecuencyMR.root.20230118.093834.974091/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar6923984540220155432/] [] /tmp/streamjob5478148704942588121.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_167

## 3.- Muestra la clasificación de temporada 2021/2022 de La Liga pero únicamente de los puntos obtenidos como visitante.

En [esta Web](https://resultados.as.com/resultados/futbol/primera/2021_2022/clasificacion/) puedes consultar cuántos puntos obtuvo cada equipo fuera de casa.

Empezamos descargando el fichero de resultados de la temporada 2021/2022 y renombrándolo a `laliga2122.csv`.

In [76]:
! wget -O laliga2122.csv https://www.football-data.co.uk/mmz4281/2122/SP1.csv

--2023-01-18 10:39:47--  https://www.football-data.co.uk/mmz4281/2122/SP1.csv
Resolving www.football-data.co.uk (www.football-data.co.uk)... ::ffff:217.160.0.246, 217.160.0.246
Connecting to www.football-data.co.uk (www.football-data.co.uk)|::ffff:217.160.0.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172174 (168K) [text/csv]
Saving to: ‘laliga2122.csv’

laliga2122.csv      100%[===================>] 168.14K  1.07MB/s    in 0.2s    

2023-01-18 10:39:47 (1.07 MB/s) - ‘laliga2122.csv’ saved [172174/172174]



Se espera este resultado:

![solución 3](./img/3.png)

In [77]:
%%writefile laligaMR.py
#!/usr/bin/python3

from mrjob.job import MRJob
from mrjob.step import MRStep
    
class LaLigaMR(MRJob):
        
    # Mapper: En esta etapa aún no hay clave (_), el valor lo recibimos en la variable line
    def mapper_points(self, _, line):
        #Por cada línea, esta se divide en los campos que forman las columnas
        _, _, _, home_team, away_team, _, _, result, *rest = line.split(',')
        
        # Si es la cabecera no emitimos nada
        if away_team == "AwayTeam":
            return
        
        if result == 'D':            
            yield away_team, 1
        if result == 'A':            
            yield away_team, 3
  

            
    def combiner_points(self, team, points):
        yield team, sum(points)
            
    def reducer_points(self, team, points):
        yield None, (team, sum(points))
        
    def reducer_classification(self, _, points):
        yield None, sorted(points, key=lambda t: t[1], reverse=True)
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper_points,
                   combiner=self.combiner_points,
                   reducer=self.reducer_points),
            MRStep(reducer=self.reducer_classification)
        ]
         
if __name__=='__main__':
    LaLigaMR.run()

Overwriting laligaMR.py


In [78]:
! python3 laligaMR.py laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/laligaMR.root.20230118.093948.807206
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/laligaMR.root.20230118.093948.807206/output
Streaming final output from /tmp/laligaMR.root.20230118.093948.807206/output...
null	[["Real Madrid", 42], ["Barcelona", 35], ["Betis", 33], ["Ath Madrid", 30], ["Sevilla", 28], ["Sociedad", 27], ["Osasuna", 25], ["Villarreal", 23], ["Valencia", 22], ["Ath Bilbao", 21], ["Celta", 21], ["Cadiz", 21], ["Granada", 16], ["Elche", 15], ["Vallecano", 13], ["Levante", 13], ["Mallorca", 12], ["Getafe", 11], ["Espanol", 9], ["Alaves", 6]]
Removing temp directory /tmp/laligaMR.root.20230118.093948.807206...


#### Comprobamos ejecución en el cluster:

In [79]:
! python3 laligaMR.py -r hadoop laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/laligaMR.root.20230118.093949.862851
uploading working dir files to hdfs:///user/root/tmp/mrjob/laligaMR.root.20230118.093949.862851/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/laligaMR.root.20230118.093949.862851/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar5400582868510656920/] [] /tmp/streamjob5953193806913837748.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_167403257393

## 4.- Muestra la diferencia de goles entre el equipo que más goles ha marcado y el que menos goles ha marcado en la temporada 2021/2022 de La Liga.

Se espera que el proceso MapReuce produzca una salida similar a la siguiente:

![solución 4](./img/4.png)

In [80]:
%%writefile laligaDMR.py
#!/usr/bin/python3

from mrjob.job import MRJob
from mrjob.step import MRStep
    
class LaLigaDMR(MRJob):
        
    # Mapper: En esta etapa aún no hay clave (_), el valor lo recibimos en la variable line
    def mapper_goals(self, _, line):
        #Por cada línea, esta se divide en los campos que forman las columnas
        _, _, _, home_team, away_team, home_goals, away_goals, result, *rest = line.split(',')
        
        # Si es la cabecera no emitimos nada
        if home_team == "HomeTeam":
            return

        if result == 'D': 
            yield home_team, (int(home_goals))
            yield away_team,(int(away_goals))
        elif result == 'H':
            yield home_team, (int(home_goals))
            yield away_team,(int(away_goals))
        else:            
            yield away_team, (int(away_goals))
            yield home_team,(int(home_goals))

            
    def combiner_goals(self, team, goals):
        yield team, sum(goals)
            
    def reducer_goals(self, team, goals):
        yield None, (team, sum(goals))
        
    def reducer_goals_classification(self,goals, teams):
        goals= sorted(teams, key=lambda t: t[1], reverse=True)
        goalsFirst = goals[0][0]
        goalsLast = goals[-1][0]
        goalsDiff=goals[0][1]-goals[-1][1]
        
        str_one = goalsFirst + " vs "+ goalsLast
        str_two = "diferencia de goles "+ str(goalsDiff)
        
        yield  (str_one , str_two)
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper_goals,
                   combiner=self.combiner_goals,
                   reducer=self.reducer_goals),
            MRStep(reducer=self.reducer_goals_classification)
        ]

         
if __name__=='__main__':
    LaLigaDMR.run()

Overwriting laligaDMR.py


In [81]:
! python3 laligaDMR.py laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/laligaDMR.root.20230118.094057.042696
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/laligaDMR.root.20230118.094057.042696/output
Streaming final output from /tmp/laligaDMR.root.20230118.094057.042696/output...
"Real Madrid vs Alaves"	"diferencia de goles 49"
Removing temp directory /tmp/laligaDMR.root.20230118.094057.042696...


#### Comprobamos ejecución en el clúster

In [82]:
! python3 laligaDMR.py -r hadoop laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/laligaDMR.root.20230118.094058.313439
uploading working dir files to hdfs:///user/root/tmp/mrjob/laligaDMR.root.20230118.094058.313439/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/laligaDMR.root.20230118.094058.313439/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar6958501444550979084/] [] /tmp/streamjob8163456734948248855.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_167403257

## 5.- Calcula la racha de los últimos cinco partidos de cada equipo en la clasificación final de La Liga en la temporada 2021/2022.

[Observa](https://www.google.com/search?q=clasificacion+liga+2021+2022&oq=clasificacion+liga+2021+2022#sie=lg) que las últimas columnas de la clasificación muestran cuál ha sido el resultado de los últimos 5 partidos de cada equipo.

![clasificacion](./img/clasificacion.png)

Se trata de que muestres la clasificación final junto con los resultados de los últimos 5 partidos. Este ejercicio es un poco más difícil y laborioso que los otros. Si usas `mrjob` probablemente te sea útil utilizar [ordenación secundaria por valor](https://mrjob.readthedocs.io/en/latest/job.html#secondary-sort), aunque también se puede resolver sin hacer uso de ella.

Se espera este resultado:

![solución 5](./img/5.png)

In [83]:
%%writefile laligaLast5MR.py
#!/usr/bin/python3

from mrjob.job import MRJob
from mrjob.step import MRStep
from datetime import datetime
    
class laligaLast5MR(MRJob):
    
    SORT_VALUES = True
        
    # Mapper: En esta etapa aún no hay clave (_), el valor lo recibimos en la variable line
    def mapper_points(self, _, line):
        #Por cada línea, esta se divide en los campos que forman las columnas
        _, date, _, home_team, away_team, _, _, result, *rest = line.split(',')
        
        # Si es la cabecera no emitimos nada
        if home_team == "HomeTeam":
            return
        
        date = datetime.strptime(date, "%d/%m/%Y").strftime("%Y/%m/%d")

        if result == 'D':            
            yield home_team, (date, 1)
            yield away_team, (date, 1)
        elif result == 'H':
            yield home_team, (date, 3)
            yield away_team, (date, 0)
        else:
            yield home_team, (date, 0)
            yield away_team, (date, 3)
            
    def reducer_points(self, team, points):
        points = list(points)
        points = [p for date, p in points]
        five_latest_points = points[-5:]
        five_latest_points.reverse()
        yield None, (team, sum(points), five_latest_points)
    
    
    def reducer_classification(self, _, points):
            yield None, sorted(points, key=lambda t: t[1], reverse=True)
            
    def steps(self):
        return [
            MRStep(mapper=self.mapper_points, reducer=self.reducer_points),
            MRStep(reducer=self.reducer_classification)
        ]
         
if __name__=='__main__':
    laligaLast5MR.run()

Overwriting laligaLast5MR.py


In [84]:
! python3 laligaLast5MR.py laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/laligaLast5MR.root.20230118.094212.200790
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/laligaLast5MR.root.20230118.094212.200790/output
Streaming final output from /tmp/laligaLast5MR.root.20230118.094212.200790/output...
null	[["Real Madrid", 86, [1, 1, 3, 0, 3]], ["Barcelona", 73, [0, 1, 3, 3, 3]], ["Ath Madrid", 71, [3, 1, 3, 3, 0]], ["Sevilla", 70, [3, 1, 1, 1, 1]], ["Betis", 65, [1, 3, 3, 0, 1]], ["Sociedad", 62, [0, 3, 3, 0, 1]], ["Villarreal", 59, [3, 0, 3, 1, 0]], ["Ath Bilbao", 55, [0, 3, 0, 1, 3]], ["Valencia", 48, [3, 1, 0, 1, 1]], ["Osasuna", 47, [0, 0, 1, 1, 1]], ["Celta", 46, [0, 3, 0, 3, 1]], ["Elche", 42, [3, 0, 0, 0, 1]], ["Espanol", 42, [1, 1, 0, 1, 0]], ["Vallecano", 42, [0, 0, 0, 1, 1]], ["Cadiz", 39, [3, 1, 0, 3, 1]], ["Getafe", 39, [0, 1, 1, 1, 1]], ["Mallorca", 39, [3, 3, 1, 0, 0]], ["Granada", 38, [1, 0, 3, 3, 1]], ["Leva

#### Comprobamos este último ejercicio en el clúster:

In [85]:
! python3 laligaLast5MR.py -r hadoop laliga2122.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/laligaLast5MR.root.20230118.094213.447892
uploading working dir files to hdfs:///user/root/tmp/mrjob/laligaLast5MR.root.20230118.094213.447892/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/laligaLast5MR.root.20230118.094213.447892/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar2785827110056456484/] [] /tmp/streamjob6531962679971207225.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.20.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/j